In [1]:
from pytorch_lightning import Trainer

import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import os
import torchvision.models as models
import torch.nn as nn
import torch
import torch.nn.functional as F
import argparse
import pytorch_lightning as pl
import sklearn 
from sklearn.metrics import confusion_matrix, auc, roc_curve
import torch
import pandas as pd
#import seaborn as sns
import pickle
from py_jotools import slurm, cache
import numpy as np
import gc
#from evalscript import eval_testset, eval_forbwtfwt

import hashlib
import dill
import active_catinous.utils as cutils

from sklearn.metrics import mean_absolute_error

from models.AgePredictor import EncoderRegressor
from models.unet3d import EncoderModelGenesis

from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib
from glob import glob
import datetime 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
%load_ext autoreload
%autoreload 

ModuleNotFoundError: No module named 'active_catinous'

In [ ]:
def ap_model_seeds(hparams, seed=2314134, split='test'):
    recalls = {'ges': [], 'geb': [], 'sie': []}
    precision = {'ges': [], 'geb': [], 'sie': []}
    
    hparams['seed'] = seed

    model, logs, df_mem, expname = lungmodel.trained_model(hparams, train=False)
    model.eval()

    for res in ['ges', 'geb', 'sie']:
        ds_test = LIDCDataset('/project/catinous/lunadata/luna_lunacombined_dataset_malignancy.csv', 
                      cropped_to=(288, 288), split=[split], res=res, validation=True)

        device = torch.device('cuda')
        iou_thres = 0.5

        overall_true_pos = dict()
        overall_false_pos = dict()
        overall_false_neg = dict()
        overall_boxes_count = dict()
        for k in np.arange(0.0, 1.01, 0.05):
            overall_true_pos[k] = 0
            overall_false_pos[k] = 0
            overall_false_neg[k] = 0
            overall_boxes_count[k] = 0
            

        for batch in ds_test:
            img_batch, annot, res, image = batch
            img_batch = img_batch.to(device)

            out = model.model(img_batch)
            out_boxes = [cutils.filter_boxes_area(out[i]['boxes'].cpu().detach().numpy(), out[i]['scores'].cpu().detach().numpy()) for i in range(len(out))]
            boxes_np = [b[0] for b in out_boxes]
            scores_np = [b[1] for b in out_boxes]

            final_boxes, final_scores = cutils.correct_boxes(boxes_np, scores_np)

            gt = annot['boxes'][0]
            for k in np.arange(0.0, 1.01, 0.05):
                false_positives = 0
                false_negatives = 0
                true_positives = 0
                detected = False
                boxes_count = 0
                if len(final_boxes)>0:
                    for i, b in enumerate(final_boxes):
                        if final_scores[i]>k:
                            boxes_count += 1
                            if cutils.bb_intersection_over_union(gt, b)>iou_thres:
                                detected = True
                            else:
                                false_positives += 1
                    if detected:
                        true_positives += 1
                    else:
                        false_negatives += 1
                overall_true_pos[k] += true_positives
                overall_false_pos[k] += false_positives
                overall_false_neg[k] += false_negatives
                overall_boxes_count[k] += boxes_count
        for k in np.arange(0.0, 1.01, 0.05):
            if (overall_false_neg[k]+overall_true_pos[k])==0:
                recalls[res].append(0.0)
            else:
                recalls[res].append(overall_true_pos[k]/(overall_false_neg[k]+overall_true_pos[k]))
            if (overall_false_pos[k]+overall_true_pos[k])==0:
                precision[res].append(0.0)
            else:
                precision[res].append(overall_true_pos[k]/(overall_false_pos[k]+overall_true_pos[k]))
    return recalls, precision

def get_ap_for_res(hparams, seed=2314134, split='test'):
    recalls, precision = ap_model_seeds(hparams, seed, split)
    
    aps = dict()
    for res in ['ges', 'geb', 'sie']:
        prec = np.array(precisions[res])
        rec = np.array(recalls[res])
        ap = []
        for t in np.arange(0.0, 1.01, 0.1):
            prec_arr = prec[rec>t]
            if len(prec_arr)==0:
                ap.append(0.0)
            else:
                ap.append(prec_arr.max())
        aps[res] = np.array(ap).mean()
    return aps